<a href="https://colab.research.google.com/github/nnilayy/Yolact/blob/main/Yolact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CLONING THE YOLACT REPO

In [ ]:
# Make sure we're in the top folder
# %cd /content

# Clone the repo
!git clone https://github.com/dbolya/yolact.git

##DCNv2
DCNv2 is needed for making Yolact work

In [ ]:
# Change to the right directory
%cd /content/yolact/external/DCNv2

# Build DCNv2
!python setup.py build develop

##PRETRAINED WEIGHTS
Getting pretrained weights from a repo


In [ ]:
# Make sure we're in the top folder
# %cd /content

# Clone the repo
# !git clone https://github.com/chentinghao/download_google_drive.git

# Create a new directory for the pre-trained weights
# !mkdir -p /content/yolact/weights

# Download the file
!python ./download_google_drive/download_gdrive.py 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP ./yolact/weights/yolact_plus_resnet50_54_800000.pth

./download_google_drive/download_gdrive.py:50: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(sys.argv) is not 3:
32.0kB [00:00, 104MB/s]


In [ ]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)                

In [ ]:
file_id = '1yp7ZbbDwvMiFJEq4ptVKTYTI2VeRDXl0'
destination = './yolact/weights/yolact_resnet50_54_800000.pth'
download_file_from_google_drive(file_id, destination) 


##CUSTOM DATASET
* Drop the file with your custom dataset in the /content by dragging and dropping as a zip folder.
*Unzip the zip folder as follows:
```
!unzip -q ../*custom_dataset*.zip -d ../
```
*Click on the following config file link: /content/yolact/data/config.py.
*Under this file, go to "DATASETS" and add the following. 
```
my_custom_dataset = dataset_base.copy({
    'name': 'My Dataset',

    'train_images': 'path_to_training_images',
    'train_info':   'path_to_training_annotation',

    'valid_images': 'path_to_validation_images',
    'valid_info':   'path_to_validation_annotation',

    'has_gt': True,
    'class_names': ('my_class_id_1', 'my_class_id_2', 'my_class_id_3', ...)
})
```
*In the above, add the path to your train and test dataset and annotations files, from the custom dataset you unziped. 



##TRAINING THE MODEL

In [ ]:
!python /content/yolact/train.py --config= yolact_base_config

In [ ]:
# Trains using the base config with a batch size of 8 (the default).
# !python /content/yolact/train.py --config=yolact_base_config

# Trains yolact_base_config with a batch_size of 5. For the 550px models, 1 batch takes up around 1.5 gigs of VRAM, so specify accordingly.
# !python /content/yolact/train.py --config=yolact_base_config --batch_size=5

# Resume training yolact_base with a specific weight file and start from the iteration specified in the weight file's name.
# !python /content/yolact/train.py --config=yolact_base_config --resume=weights/yolact_base_10_32100.pth --start_iter=-1

# Use the help option to see a description of all available command line arguments
# !python /content/yolact/train.py --help

In [ ]:
# Pre-train the image based model
# !python train.py --config=yolact_edge_youtubevis_config

# Train the flow (warping) module
# !python train.py --config=yolact_edge_vid_trainflow_config --resume=./weights/yolact_edge_youtubevis_847_50000.pth

# Fine tune the network jointly
# !python train.py --config=yolact_edge_vid_config --resume=./weights/yolact_edge_vid_trainflow_144_100000.pth


##TESTING THE MODEL

In [ ]:
!python /content/yolact/eval.py --trained_model=/content/yolact/weights/yolact_resnet50_54_800000.pth --config=yolact_base_config --score_threshold=0.15 --top_k=15 --display --image=/content/tennis.jpg

# Display qualitative results on the specified image: --image=my_image.png 
# Process a whole folder of images: --images=path/to/input/folder:path/to/output/folder 
# Process an image and save it to another file: --image=input_image.png:output_image.png 

# Display a video in real-time. "--video_multiframe" will process that many frames at once for improved performance.
# If you want, use "--display_fps" to draw the FPS directly on the frame.
# --video_multiframe=4 --video=my_video.mp4 

# Display a webcam feed in real-time. If you have multiple webcams pass the index of the webcam you want instead of 0.
# --video_multiframe=4 --video=0 

# Process a video and save it to another file. This uses the same pipeline as the ones above now, so it's fast!
# --video_multiframe=4 --video=input_video.mp4:output_video.mp4 


/usr/local/lib/python3.8/dist-packages/torch/jit/_recursive.py:256: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
/usr/local/lib/python3.8/dist-packages/torch/jit/_recursive.py:256: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
/usr/local/lib/python3.8/dist-packages/torch/jit/_recursive.py:256: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
Loading model...Traceback (most recent call last):
  File "/content/yolact/eval.py", line 1098, in <module>
    net.load_weights(args.trained_model)
  File "/content/yolact/yolact.py", line 479, in load_weights
    state_dict = torch.load(

##DISPLAYING THE RESULTS 


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

output_images = Path('output_images')

def show_image(img_path):
  img = cv2.imread(img_path)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(16,16))
  plt.imshow(img_cvt)
  plt.show()

# Iterate through all of the output images and display them
for img_path in output_images.iterdir():
  print(img_path)
  show_image(str(img_path))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive
!mkdir /content/yolact/new_weigths
!cp /content/drive/MyDrive/yolact_plus_resnet50_54_800000.pth /content/yolact/new_weigths/yolact_plus_resnet50_54_800000
